[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/me-manu/gammaALPs/blob/master/docs/tutorials/cta_spectrum_simulation_ngc1275.ipynb)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
from astropy import constants as c
from scipy.integrate import simpson


# Simulating NGC 1275 with and without axion-like particles

First we will simulate photon-survival probabilities for the propagation from NGC1275 inside the Perseus cluster towards Earch. Secondly, we will simulate a CTA observation of NGC1275 without ALPs and fit this simulation with our ALP modified spectra.

## Prerequisites

- installed `gammapy`, version 1.0.1, see https://docs.gammapy.org/1.0/index.html
- installed `gammaALPs` version 0.3, see https://gammaalps.readthedocs.io/en/latest/ (will also be done below)
- CTA IRFs available. We will use the CTA North IRFs for 20 degree zenith observation with backgrounds optimized for 5 hour observations. The IRFs are available here: https://zenodo.org/record/5499840#.YUya5WYzbUI

## Further information

This hands-on sesssion roughly follows the tutorials given here:
- https://docs.gammapy.org/1.0/tutorials/analysis-1d/spectrum_simulation.html
- https://docs.gammapy.org/1.0/tutorials/analysis-1d/spectral_analysis.html
- https://gammaalps.readthedocs.io/en/latest/tutorials/mixing_ICM_Gaussian_Turbulence.html


### Installing `gammapy` from within notebook

If you haven't installed gammapy yet, you can run the following command. If you are working in your machine and not on google colab, I strongly advice that you set up a new conda environment and not simply run `pip`. 

In [ ]:
pip install gammapy==1.0.1

### Downloading CTA IRFs

If you haven't downloaded the IRFs yet, you can simply run the command below in google colab. 

In [ ]:
!wget https://zenodo.org/record/5499840/files/cta-prod5-zenodo-fitsonly-v0.1.zip

Then you can unzip the package:

In [ ]:
!unzip cta-prod5-zenodo-fitsonly-v0.1.zip

And look at the contents:

In [ ]:
!ls fits/

We will work with the CTA North IRFs for 20 degree zenith observations. So we have to untar the corresponding file:

In [ ]:
!tar -xzvf fits/CTA-Performance-prod5-v0.1-North-20deg.FITS.tar.gz

In [ ]:
!ls

## Installing `gammaALPs`

We assume that `gammapy` version 1.0.1 is already installed and this notebook runs with the corresponding kernel. If not already installed, the `gammaALPs` package can be installed with the following command.

In [ ]:
!pip install gammaALPs==0.3

## Calculating the photon survival probabilities

First, we calculate the survival probabilities $P_{\gamma\to\gamma}$ for mixing inside the Perseus cluster and in the magnetic field of the milky way. We will also include absorption by the EBL. The modelling of the cluster magnetic field follows Ajello et al. (2016) (https://arxiv.org/abs/1603.06978) where the $B$ field is modelled as a field with Gaussian turbulence. 

First, we need some imports from `gammaALPs`:

In [ ]:
from gammaALPs.core import Source, ALP, ModuleList
from gammaALPs.base import environs, transfer
from ebltable.tau_from_model import OptDepth

And initialize an ALP object, which stores the ALP mass $m_a$ (in neV) and the coupling $g_{a\gamma}$ (in $10^{-11}\mathrm{GeV}^{-1}$).

In [ ]:
m, g = 1.,1.
alp = ALP(m,g)

Next, we set the source properties (redshift and sky coordinates) in the ```Source``` container. These can be taken form your favorite catalot for extragalactic objects, e.g., <a href="http://ned.ipac.caltech.edu/">NED</a>.

In [ ]:
ngc1275 = Source(z=0.017559, ra='03h19m48.1s', dec='+41d30m42s')
print (ngc1275.z)
print (ngc1275.ra, ngc1275.dec)
print (ngc1275.l, ngc1275.b)

### Init the module list

Next, we can initialize the list of transfer modules that will store the different magnetic field environments. 
First, we provide the energies for which we would like to compute the conversion probability

In [ ]:
EGeV = np.logspace(1.5, 4.5, 150)

pin = np.diag((1.,1.,0.)) * 0.5Now initialize the initial photon polarization. Since we are dealing with a gamma-ray source, no ALPs are initially present in the beam (third diagonal element is zero). The polarization density matrix is normalized such that its trace is equal to one, $\mathrm{Tr}(\rho_\mathrm{in}) = 1$.

In [ ]:
pin = np.diag((1.,1.,0.)) * 0.5

The module list is initialized with our choices for the ALP, our source, the initial polarization, and the energies at which we compute the photon-ALP mixingm

In [ ]:
ml = ModuleList(alp, ngc1275, pin = pin, EGeV = EGeV)

Now, we add the different environments to the module list. For the EBL, we will use the model from [Dominguez et al. (2011)](https://ui.adsabs.harvard.edu/abs/2011MNRAS.410.2556D/abstract). The other parameters are the same in [Ajello et al. 2016](https://arxiv.org/abs/1603.06978). 

In [ ]:
ml.add_propagation("ICMGaussTurb", 
                  0, # position of module counted from the source. 
                  nsim=100, # number of random B-field realizations
                  B0=10.,  # rms of B field in muG
                  n0=3.9e-2,  # normalization of electron density in cm-3
                  n2=4.05e-3, # second normalization of electron density, see Churazov et al. 2003, Eq. 4 on cm-3
                  r_abell=500., # extension of the cluster in kpc
                  r_core=80.,   # electron density parameter, see Churazov et al. 2003, Eq. 4 in kpc
                  r_core2=280., # electron density parameter, see Churazov et al. 2003, Eq. 4 in kpc
                  beta=1.2,  # electron density parameter, see Churazov et al. 2003, Eq. 4
                  beta2=0.58, # electron density parameter, see Churazov et al. 2003, Eq. 4
                  eta=0.5, # scaling of B-field with electron denstiy
                  kL=0.18, # maximum turbulence scale in kpc^-1, taken from A2199 cool-core cluster, see Vacca et al. 2012 
                  kH=9.,  # minimum turbulence scale, taken from A2199 cool-core cluster, see Vacca et al. 2012
                  q=-2.80, # turbulence spectral index, taken from A2199 cool-core cluster, see Vacca et al. 2012
                  seed=0 # random seed for reproducability, set to None for random seed.
                 )
ml.add_propagation("EBL",1, eblmodel='dominguez') # EBL attenuation comes second, after beam has left cluster
ml.add_propagation("GMF",2, model='jansson12') # finally, the beam enters the Milky Way Field

List the module names:

In [ ]:
print(ml.modules.keys())

We can also inspect the magnetic-field realization and electron density along the line of sight. The magnetic-field realizations are stored in `ml.modules["ICMGaussTurb"].Bn`,

In [ ]:
print (ml.modules["ICMGaussTurb"].Bn.shape)

And multiplying the magnetic field with the $\psi$ angles stored in `ml.modules["ICMGaussTurb"].Psin` will give us the two components transversal to the propagation direction:

In [ ]:
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i, B in enumerate(ml.modules["ICMGaussTurb"].Bn):
    ax1.plot(ml.modules["ICMGaussTurb"].r,
             B * np.sin(ml.modules["ICMGaussTurb"].psin[i]),
             lw=1 if not i else 0.1,
             alpha=1 if not i else 0.1,
             color=plt.cm.tab10(0.)
            )
    ax2.plot(ml.modules["ICMGaussTurb"].r,
             B * np.cos(ml.modules["ICMGaussTurb"].psin[i]),
             lw=1 if not i else 0.1,
             alpha=1 if not i else 0.1,
             color=plt.cm.tab10(0.1)
            ) 
    
ax1.set_ylabel('$B$ field ($\mu$G)')
ax1.set_xlabel('$r$ (kpc)')
ax2.set_xlabel('$r$ (kpc)')

The coherent magnetic field in the Milky Way used in the mixing can be plotted in a similar way. Note that we're using the attributes `ml.modules["GMF"].B` and `ml.modules["GMF"].psi` since we are dealing with one coherent magnetic field and not many random realizations. 

In [ ]:
plt.plot(ml.modules["GMF"].r, ml.modules["GMF"].B * np.sin(ml.modules["GMF"].psi),
         lw=2)
plt.plot(ml.modules["GMF"].r, ml.modules["GMF"].B * np.cos(ml.modules["GMF"].psi),
         lw=2) 
plt.ylabel('$B$ field ($\mu$G)')
plt.xlabel('$r$ (kpc)')

The electron density in our Perseus cluster model looks like this (and comes from [Churazov et al., 2003](https://ui.adsabs.harvard.edu/abs/2003ApJ...590..225C/abstract))

In [ ]:
plt.loglog(ml.modules["ICMGaussTurb"].r, ml.modules["ICMGaussTurb"].nel)
plt.ylabel('$n_\mathrm{el}$ (cm$^{-3}$)')
plt.xlabel('$r$ (kpc)')

### Spatial correlation and coherence length

The `gammaALPs.bfields.Bgaussian` class has methods to calculate the spatial correlation of the magnetic field and the rotation measure. We can access these methods through the the magnetic field model mehtod, which we can access through `ml.modules['ICMGaussTurb'].Bfield_model`. 

The spatial correlation $C(x_3) = \langle B_\perp(\vec{x}) B_\perp(\vec{x} + x_3 \vec{e}_3)\rangle$ of the transversal magnetic field along the line of sight $z$ is computed like this:

In [ ]:
x3 = np.linspace(0.,50.,1000)  # distance in kpc from cluster center
c = ml.modules["ICMGaussTurb"].Bfield_model.spatial_correlation(x3) 

plt.plot(x3, c / c[0])
plt.xlabel("$z$ (kpc)")
plt.ylabel("$C(z) / C(0)$")
plt.grid(True)

This is turn can be used to calculate the coherence length of the field, 
$$ \Lambda_C = \frac{1}{C(0)} \int\limits_0^\infty C(z)dz. $$


In [ ]:
z = np.linspace(0.,1e3,1000)  # distance in kpc from cluster center
c = ml.modules["ICMGaussTurb"].Bfield_model.spatial_correlation(z) 

Lambda_c = simpson(c, z) / c[0]

print ("Coherence length of the field is Lambda_C = {0:.3e} kpc".format(Lambda_c))

### Run all modules

Now we run the modules. 

The ```px, py, pa``` variables contain the mixing probability into the two photon polarization states ($x$, $y$) and into the axion state ($a$). If this is taking too much time, consider reducing the number of energies and / or the number of simulated $B$-field realizations. On my laptop, 100 realizations at 150 energies take around 30s. On google colab, it took around 2 minutes.

We can also change the ALP parameters before running the modules:

In [ ]:
ml.alp.m = 30.
ml.alp.g = 0.5

In [ ]:
%%time
px, py, pa = ml.run()

### Plot the output 

We now plot the resulting total survival probability, $P_{\gamma\gamma} = P_x + P_y$ for each 
magnetic-field realization.

In [ ]:
pgg = px + py # the total photon survival probability

effect = dict(path_effects=[withStroke(foreground="w", linewidth=2)])

for i, p in enumerate(pgg):  # plot all realizations
    plt.loglog(ml.EGeV, p, color="C0",
               alpha=1 if not i else 0.2,
               lw=1 if not i else 0.1)
    
# plot the EBL case only
atten = np.exp(-ml.modules['OptDepth'].opt_depth(ml.source.z, ml.EGeV / 1e3))
plt.loglog(ml.EGeV, atten, ls='--', color="C1")


plt.xlabel('Energy (GeV)')
plt.ylabel('Photon survival probability')

plt.annotate(r'$m_a = {0:.1f}\,\mathrm{{neV}}, g_{{a\gamma}}'
             r' = {1:.1f} \times 10^{{-11}}\,\mathrm{{GeV}}^{{-1}}$'.format(ml.alp.m, ml.alp.g),
             xy=(0.05,0.1), 
             size ='x-large',
             xycoords='axes fraction',
             **effect)

plt.ylim(1e-1, 1.1)

### Questions:

- how does $P_{\gamma\to\gamma}$ change for different ALP masses / couplings? You may want to change one parameter at a time and simulate this for less B-field realizations. 
- how does the $B$-field correlation length and $P_{\gamma\to\gamma}$ change for different $k_{L/H}$ and power-law index $q$ and rms field strength $B$?

## `Gammapy` simulations

Now that we have our theoretical predictions for the photon propagation with ALPs (stored in the `pgg` array) and without ALPs (EBL only case stored in the `atten` array), we can setup and run the `gammapy` simulation.

First we need some additional imports. 

In [ ]:
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion
import operator

And more imports from `gammapy`:

In [ ]:
from gammapy.data import Observation, observatory_locations
from gammapy.datasets import SpectrumDataset, SpectrumDatasetOnOff
from gammapy.irf import load_cta_irfs
from gammapy.makers import SpectrumDatasetMaker
from gammapy.maps import MapAxis, RegionGeom
from gammapy.modeling import Fit
from gammapy.modeling import models 
from gammapy.estimators import FluxPointsEstimator

## Simulation of a single spectrum

To do a simulation, we need to define the observational parameters like
the livetime, the offset, the assumed integration radius, the energy
range to perform the simulation for and the choice of spectral model. We
then use an in-memory observation which is convolved with the IRFs to
get the predicted number of counts. This is Poission fluctuated using
the `fake()` to get the simulated counts for each observation.




In [ ]:
# Define simulation parameters parameters
livetime = 5. * u.h

pointing = SkyCoord(ml.source.ra, ml.source.dec, unit="deg", frame="fk5")
offset = 0.5 * u.deg

# Reconstructed and true energy axis
energy_axis = MapAxis.from_edges(
    np.logspace(-1, 1.5, 26), unit="TeV", name="energy", interp="log"
)

energy_axis_true = MapAxis.from_edges(
    np.logspace(-1.5, 2.0, 71), unit="TeV", name="energy_true", interp="log"
)

on_region_radius = Angle("0.11 deg")

center = pointing.directional_offset_by(position_angle=0 * u.deg, separation=offset)
on_region = CircleSkyRegion(center=center, radius=on_region_radius)

Now we define the intrinsic source model. This is taken from the VERITAS observation of NGC1275 during a flaring episode. During the observation, the source could be described with a power law with exponential cutoff, 

$$ \phi(E) = N_0 \left(\frac{E}{E_0}\right)^{-\Gamma}\exp\left(-\frac{E}{E_\mathrm{cut}}\right) $$

with $N_0 = 1.54\times10^{-9}\mathrm{cm}^{-2}\,\mathrm{s}^{-1}\,\mathrm{TeV}^{-1}$, $\Gamma = 2.11$, $E_\mathrm{cut} = 0.54\,$TeV, and $E_0 = 0.3\,$TeV.

Go to the available spectral models here: https://docs.gammapy.org/1.0/user-guide/model-gallery/index.html 
and implement the correct spectrum. Note that the `gammapy` implementation uses $\lambda = E_\mathrm{cut}^{-1}$ for the fitting. Also note that the parameters need to set with the correct units (using the `astropy` implementation). 


In [ ]:
model_intrinsic = models.ExpCutoffPowerLawSpectralModel(
    index=2.11,
    amplitude=1.54e-9 * u.Unit("cm-2 s-1 TeV-1"),
    lambda_=1. / 0.54 / u.TeV,
    reference=0.3 * u.TeV,
)
print(model_intrinsic)

With the intrinsic model, we now build a compound model, which multiplies the intrinsic model with a template model that includes the EBL absorption. 

In [ ]:
# EBL absorption template
ebl_absorption = models.TemplateSpectralModel(ml.EGeV * u.GeV, atten)

# Compound model
model_input = models.CompoundSpectralModel(model_intrinsic, ebl_absorption, operator=operator.mul)
print(model_input)

# and the Skymodel
model_no_alps = models.SkyModel(spectral_model=model_intrinsic, name="ngc1275")

Let's plot the intrinsic and observed models:

In [ ]:
ax = model_intrinsic.plot(energy_bounds=[0.1, 30.] * u.TeV, energy_power=2, ls='--')
_ = model_input.plot(energy_bounds=[0.1, 30.] * u.TeV, energy_power=2)

We see that the EBL absorption is almost negligible due to the small value of the redshift. 

### Load the instrumental response function

You should have downloaded and unpacked the CTA response functions, which are available here: https://zenodo.org/record/5499840


In the cell below, you need to set the correct path with the IRF for 20 deg zenith and 5 hours observation time. 

In [ ]:
irfs = load_cta_irfs(
    "./Prod5-North-20deg-AverageAz-4LSTs09MSTs.18000s-v0.1.fits.gz"
)

location = observatory_locations["cta_north"]
obs = Observation.create(
    pointing=pointing,
    livetime=livetime,
    irfs=irfs,
    location=location,
)
print(obs)

#### Peek at the IRFs

We check the effective area, PSF, and energy dispersion matrix.

In [ ]:
irfs["aeff"].peek()

In [ ]:
irfs["edisp"].peek()

In [ ]:
irfs["psf"].peek()

### Simulate a spectra

Now we have to follow some steps to simulate the On/Off data set.


In [ ]:
# Make the SpectrumDataset
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name="obs-0"
)
maker = SpectrumDatasetMaker(selection=["exposure", "edisp", "background"])

dataset = maker.run(dataset_empty, obs)

# Set the model on the dataset, and fake
dataset.models = model_no_alps
dataset.fake(random_state=42)
print(dataset)

In [ ]:
dataset.peek()

You can see that background counts are now simulated.

### On-Off analysis

To do an on off spectral analysis, which is the usual science case, the
standard would be to use `SpectrumDatasetOnOff`, which uses the
acceptance to fake off-counts.
The `acceptance_off` is essentially the parameter $\alpha^{-1}$. 

In [ ]:
dataset_on_off = SpectrumDatasetOnOff.from_spectrum_dataset(
    dataset=dataset, acceptance=1, acceptance_off=5
)
dataset_on_off.fake(npred_background=dataset.npred_background(), random_state=42)
print(dataset_on_off)

In [ ]:
dataset_on_off.peek()

### Look at the statistics of the dataset

Basic statistics of the dataset like On, Off, excess counds, and the source significance are available from the `info_dict`.

In [ ]:
dataset_on_off.info_dict()

### Perform the fit

For our baseline model, without ALPs, we fit the observation with the same model that we used to simulate the data set, i.e., our intrinsic model with EBL absorption. 

In [ ]:
# init the fit
fit = Fit(optimize_opts={"print_level":2})

In [ ]:
%%time
result_no_alps = fit.run(dataset_on_off)

In [ ]:
# check the best-fit parameters
result_no_alps.models[0].spectral_model.parameters.to_table()

The Cash statistic, $C = -2\ln\mathcal{L}$, is stored in `results_no_alps.total_stat`:

In [ ]:
result_no_alps.total_stat

One can easily plot the excess counts and the predicted signal counts:

In [ ]:
plt.figure()
ax_spectrum, ax_residuals = dataset_on_off.plot_fit()
dataset_on_off.plot_masks(ax=ax_spectrum)

Before performing the fit with ALPs, we generate flux points, so that we can plot the SED.

In [ ]:
fpe = FluxPointsEstimator(
    energy_edges=energy_axis.edges, reoptimize=False, selection_optional=["errn-errp", "ul"], source="ngc1275"
)
flux_points = fpe.run(datasets=dataset_on_off)

Plot the flux points, define the energy range to flux points with a $\mathrm{TS}$ values larger than 1

In [ ]:
# TS values of flux points
ts_values = np.squeeze(flux_points.ts.data)

# mask 
mask_ts = ts_values > 1.

# energy range for plotting
plot_range = flux_points.energy_axis.edges[:-1][mask_ts][0] * 0.9, flux_points.energy_axis.edges[1:][mask_ts][-1]
print(plot_range)

# plot the model
ax = result_no_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")

# plot the butterfly, i.e., the model error
result_no_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the flux points
flux_points.plot(ax=ax, sed_type="e2dnde", color="darkorange")

# limit the energy range
plt.xlim(plot_range)

## Fit with Axion-like particles 

As a last step, we perform a fit with axion-like particles. For that, we loop through the 
$B$ field realizations. We first fit and plot one example. Then we perform the loop.

First we define a new compound model that includes the ALP effect. 

In [ ]:
# use the first B field realization
alps = models.TemplateSpectralModel(ml.EGeV * u.GeV, pgg[0])

# and define the compound model
model_obs_with_alps = models.CompoundSpectralModel(model_intrinsic, alps, operator=operator.mul)

model_w_alps = models.SkyModel(spectral_model=model_obs_with_alps, name="ngc1275")

Let's plot the different models

In [ ]:
ax = model_intrinsic.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2, ls='--')
_ = model_input.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2)
_ = model_obs_with_alps.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2)

Then we set the model and fit it to data

In [ ]:
dataset_on_off.models = model_w_alps

In [ ]:
result_w_alps = fit.run(dataset_on_off)

In [ ]:
# check the best-fit parameters
result_w_alps.models[0].spectral_model.parameters.to_table()

In [ ]:
result_w_alps.total_stat

#### Questions:
- What do you notice for the best-fit parameters?
- Can you plot the flux points, the best fit without ALPs and the best-fit models with ALPs?

In [ ]:
# plot the model w/o ALPs
ax = result_no_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")
result_no_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the model w/ ALPs
ax = result_w_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")
result_w_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the flux points
flux_points.plot(ax=ax, sed_type="e2dnde", color="k")

# limit the energy range
plt.xlim(plot_range)

Lastly, we loop through the magnetic field realizations and perform the fit in each step. 
We save the $C$ stat value and plot the difference, $\Delta C = C_\mathrm{w/~ALP} - C_\mathrm{no~ALP}$.

In [ ]:
%%time

tot_stat = []

fit = Fit(optimize_opts={"print_level":1})  # reduce print level

for i, p in enumerate(pgg):
    
    # model
    alps = models.TemplateSpectralModel(ml.EGeV * u.GeV, pgg[i])
    model_obs_with_alps = models.CompoundSpectralModel(model_intrinsic, alps, operator=operator.mul)
    model_w_alps = models.SkyModel(spectral_model=model_obs_with_alps, name="ngc1275")
    
    # set the model of the dataset
    dataset_on_off.models = model_w_alps
    
    # fit 
    result_w_alps = fit.run(dataset_on_off)
    
    # save the total stat value
    tot_stat.append(result_w_alps.total_stat)

In [ ]:
# convert to np.array
tot_stat = np.array(tot_stat)

In [ ]:
# plot the histogram
llr = tot_stat - result_no_alps.total_stat  # the log likelihood ratio values
bins = np.linspace(0, llr.max(), 20)  # bin edges for histogram
plt.hist(llr, bins=bins)
plt.xlabel("$\Delta C = C_\mathrm{w/~ALP} - C_\mathrm{no~ALP}$")

# and the fifth quantile
print(int(len(tot_stat) * 0.05))
tot_stat_q5 = np.sort(tot_stat)[int(len(tot_stat) * 0.05)]

plt.axvline(tot_stat_q5 - result_no_alps.total_stat, ls='--', color='r')
print(tot_stat_q5 - result_no_alps.total_stat)

### Questions:
- How do interpret this histogram?
- Which B field realization should we use to decide if the ALP hypothesis is preferred or not?
- How do the results change when you change the observation live time?
- How do the results change when you change the intrinsic model to a power law?
- How do the results change when you change the ALP mass or photon-ALP coupling?